In [1]:
import praw as praw
import pandas as pd
import os
import datetime
import re
import emoji
import spacy
from better_profanity import profanity
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import credential
import time
from prawcore.exceptions import TooManyRequests

In [2]:
nlp = spacy.load("en_core_web_sm")
analyzer = SentimentIntensityAnalyzer()
profanity.load_censor_words()

def is_question_spacy(text):
    doc = nlp(text)
    return 1 if any(token.dep_ == "aux" and token.head.pos_ == "VERB" for token in doc) else 0

def get_user_data(author):
    if not author:
        return None, None, 0  # No author data

    try:
        user_karma = author.comment_karma
        account_age = (datetime.datetime.now(datetime.UTC) - datetime.datetime.fromtimestamp(author.created_utc, datetime.UTC)).days
        has_flair = 1 if author.has_verified_email else 0
        return user_karma, account_age, has_flair
    except TooManyRequests:
        print("⚠️ Rate limit hit! Sleeping for 15 seconds...")
        time.sleep(15)  # Wait 15 seconds if rate limit is hit
        return None, None, 0
    except Exception as e:
        print(f"⚠️ Failed to fetch user data: {e}")
        return None, None, 0

In [3]:
reddit = praw.Reddit(
    client_id= credential.REDDIT_CLIENT_ID,  
    client_secret= credential.REDDIT_CLIENT_SECRET,  
    user_agent= credential.REDDIT_USER_AGENT
)

reddit.read_only = True

In [4]:
subreddit_name = "AskReddit"
sort_by = "hot"  # Options: 'hot', 'new', 'top', 'rising'
num_posts = 10

In [5]:
data = []
subreddit = reddit.subreddit(subreddit_name)
posts = getattr(subreddit, sort_by)(limit=num_posts)

for post in posts:
    submission = reddit.submission(id=post.id)
    submission.comments.replace_more(limit=1)  # Limit deep threading

    # Get post creation time in UTC
    post_time_utc = datetime.datetime.fromtimestamp(submission.created_utc, datetime.UTC)

    for comment in submission.comments.list():
        # Text features
        text = comment.body
        sentiment_score = analyzer.polarity_scores(text)["compound"]
        text_length = len(text)
        num_words = len(text.split())
        contains_question = is_question_spacy(text)
        contains_emoji = 1 if any(emoji.is_emoji(char) for char in text) else 0
        contains_profanity = 1 if profanity.contains_profanity(text) else 0

        # Metadata features
        comment_time_utc = datetime.datetime.fromtimestamp(comment.created_utc, datetime.UTC)
        comment_age_hours = (comment_time_utc - post_time_utc).total_seconds() / 3600
        is_early_comment = 1 if comment_age_hours <= 1 else 0  # Early = within 1 hour

        comment_hour = comment_time_utc.hour
        comment_day = comment_time_utc.weekday()
        subreddit_name = submission.subreddit.display_name
        parent_score = comment.parent().score if comment.parent_id != submission.id else None

        # User-based features
        user_karma, account_age, has_flair = get_user_data(comment.author)

        # Engagement variables (dependent variables)
        comment_score = comment.score
        num_replies = len(comment.replies)

        # Store data
        data.append({
            "Comment Score": comment_score,
            "Number of Replies": num_replies,
            "Text" : text,
            "Subreddit Name": subreddit_name,
            "Sentiment Score": sentiment_score,
            "Text Length": text_length,
            "Word Count": num_words,
            "Contains Question": contains_question,
            "Contains Emoji": contains_emoji,
            "Contains Profanity": contains_profanity,
            "Comment Age (hours)": comment_age_hours,
            "Comment Hour": comment_hour,
            "Comment Day": comment_day,
            "Is Early Comment": is_early_comment,
            "Parent Score": parent_score,
            "User Karma": user_karma,
            "Account Age (days)": account_age,
        })

⚠️ Rate limit hit! Sleeping for 15 seconds...
⚠️ Failed to fetch user data: 'Redditor' object has no attribute 'comment_karma'
⚠️ Rate limit hit! Sleeping for 15 seconds...
⚠️ Failed to fetch user data: 'Redditor' object has no attribute 'comment_karma'
⚠️ Rate limit hit! Sleeping for 15 seconds...


In [6]:
df_new = pd.DataFrame(data)

In [7]:
csv_file = "reddit_engagement_data.csv"

if os.path.exists(csv_file):
    df_existing = pd.read_csv(csv_file)
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    
    # Remove duplicate rows
    df_combined = df_combined.drop_duplicates(subset=["Text", "Subreddit Name"], keep="first")
    df_combined.to_csv(csv_file, index=False)
    print(f"✅ {len(df_new)} new rows checked. Unique dataset saved to '{csv_file}'.")

else:
    df_new.to_csv(csv_file, index=False)
    print(f"✅ First-time save: {len(df_new)} rows saved to '{csv_file}'.")

✅ 5801 new rows checked. Unique dataset saved to 'reddit_engagement_data.csv'.
